# Introduction
Determin Porygon DT of Sampled Users to Each Restaurant


# Data collection
We get the data from big query.

In [2]:
# load packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
#from google.colab import drive # to allow colab save file in my drive
#drive.mount('drive')
cwd=os.getcwd()
cwd

'/content'

In [3]:
query="""
select * from `fulfillment-dwh-production.temp.results_tw_7`
"""
# run the query and save the results as data frame named df
df=gbq.read_gbq(query,project_id="fulfillment-dwh-production")


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=9r2jjcwCBRPKY9BYa8dYmEIuyIy90K&prompt=consent&access_type=offline
Enter the authorization code: 4/ygH_MPus2sAMDfKosK9p1ABdRS-bFLZJT7UDy8QmKahgBvAYAJHkOBw


In [4]:
# to save money, next time we can read the csv file instead of query from bigquery
#df.to_csv("taiwan_geohash7.csv", sep="\t")
#!cp taiwan_geohash7.csv "drive/My Drive/dataset"
#df = pd.read_csv("/content/taiwan_geohash7.csv")
df.head()

,user_zone_id,geohash,geohash_center,restaurant_id,weight,porygon_dt
0,3,wsqq7fh,POINT(121.460037231445 25.0165557861328),y6oa,79.0,14
1,5,wsqqsr5,POINT(121.48063659668 25.0879669189453),e7zo,78.0,7
2,3,wsqq5zd,POINT(121.457290649414 25.0028228759766),e6mx,78.0,6
3,6,wsqqm7q,POINT(121.528701782227 25.0234222412109),e1qz,92.0,12
4,5,wsqqsej,POINT(121.494369506836 25.0659942626953),y4yq,77.0,4


In [5]:
pivot=df.pivot_table("restaurant_id", index=['geohash_center',"geohash","weight"], columns='porygon_dt', aggfunc='count')
totaldistinct=len(pivot) # total distinct geohashes
pivot = pivot.fillna(0) # replace NA with 0
pivot.head()

,,porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,,
POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0
POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0
POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0
POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0
POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0


In [6]:
pivot_cumsum = pivot.cumsum(axis=1)
pivot_cumsum.head()

,,porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
geohash_center,geohash,weight,,,,,,,,,,,,,,,,,,,
POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,6.0,10.0,20.0,36.0,67.0,101.0,141.0,166.0,190.0,204.0
POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,11.0,22.0,45.0,84.0,109.0,117.0,128.0,154.0,197.0
POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,6.0,11.0,26.0,51.0,86.0,109.0,116.0,128.0,154.0
POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,9.0,23.0,29.0,60.0,102.0,141.0,184.0,189.0,196.0
POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,11.0,25.0,32.0,67.0,108.0,150.0,180.0,189.0,204.0


# Weighted version

In [7]:
weight = pivot_cumsum
weight = weight.reset_index()
weight

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,2.0,6.0,10.0,20.0,36.0,67.0,101.0,141.0,166.0,190.0,204.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,7.0,11.0,22.0,45.0,84.0,109.0,117.0,128.0,154.0,197.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,6.0,11.0,26.0,51.0,86.0,109.0,116.0,128.0,154.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,2.0,9.0,23.0,29.0,60.0,102.0,141.0,184.0,189.0,196.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2.0,4.0,11.0,25.0,32.0,67.0,108.0,150.0,180.0,189.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,2.0,4.0,4.0,10.0,27.0,30.0,48.0,66.0,82.0,107.0,115.0,127.0,131.0,138.0,141.0,148.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,1.0,2.0,4.0,4.0,10.0,24.0,33.0,51.0,68.0,84.0,108.0,119.0,126.0,132.0,139.0,141.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,1.0,3.0,4.0,4.0,13.0,28.0,36.0,56.0,71.0,86.0,110.0,121.0,129.0,132.0,140.0,143.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,2.0,3.0,3.0,4.0,15.0,24.0,37.0,56.0,68.0,91.0,110.0,119.0,129.0,134.0,139.0,142.0


In [8]:
numOfRestaurants = [1, 5, 10, 15, 20, 25, 30]
drivingTimesinMin = range(2,21)
resArray = []
for indexNumOfRestaurants in numOfRestaurants:
    sumByDrivingTimes = []
    for indexDrivingTimesInMin in drivingTimesinMin:
        sumByDrivingTimes.append(weight.loc[weight[indexDrivingTimesInMin] >= indexNumOfRestaurants, "weight"].sum())
        
    sumByDrivingTimes.insert(0,indexNumOfRestaurants)
    resArray.append(sumByDrivingTimes)

resArray

[[1,
  283795.0,
  328087.0,
  340242.0,
  344754.0,
  346941.0,
  348018.0,
  348805.0,
  349139.0,
  349313.0,
  349392.0,
  349418.0,
  349442.0,
  349456.0,
  349463.0,
  349464.0,
  349468.0,
  349470.0,
  349470.0,
  349472.0],
 [5,
  172925.0,
  287538.0,
  320843.0,
  334152.0,
  340522.0,
  343843.0,
  345920.0,
  347260.0,
  348161.0,
  348807.0,
  349059.0,
  349277.0,
  349348.0,
  349400.0,
  349429.0,
  349444.0,
  349453.0,
  349461.0,
  349462.0],
 [10,
  98614.0,
  250095.0,
  304229.0,
  324727.0,
  334897.0,
  340644.0,
  343834.0,
  345616.0,
  347087.0,
  348002.0,
  348696.0,
  349093.0,
  349245.0,
  349352.0,
  349394.0,
  349424.0,
  349448.0,
  349455.0,
  349455.0],
 [15,
  54576.0,
  212324.0,
  287280.0,
  316135.0,
  330218.0,
  337408.0,
  342011.0,
  344224.0,
  345831.0,
  347168.0,
  347988.0,
  348687.0,
  348997.0,
  349231.0,
  349315.0,
  349389.0,
  349430.0,
  349450.0,
  349454.0],
 [20,
  31444.0,
  177331.0,
  269352.0,
  307352.0,
  325073.0,

In [9]:
colNames = np.append("Number of Restaurants", drivingTimesinMin)

df2 = pd.DataFrame(resArray,
                   columns = colNames)
df2=df2.set_index('Number of Restaurants')
df2

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Number of Restaurants,,,,,,,,,,,,,,,,,,,
1,283795.0,328087.0,340242.0,344754.0,346941.0,348018.0,348805.0,349139.0,349313.0,349392.0,349418.0,349442.0,349456.0,349463.0,349464.0,349468.0,349470.0,349470.0,349472.0
5,172925.0,287538.0,320843.0,334152.0,340522.0,343843.0,345920.0,347260.0,348161.0,348807.0,349059.0,349277.0,349348.0,349400.0,349429.0,349444.0,349453.0,349461.0,349462.0
10,98614.0,250095.0,304229.0,324727.0,334897.0,340644.0,343834.0,345616.0,347087.0,348002.0,348696.0,349093.0,349245.0,349352.0,349394.0,349424.0,349448.0,349455.0,349455.0
15,54576.0,212324.0,287280.0,316135.0,330218.0,337408.0,342011.0,344224.0,345831.0,347168.0,347988.0,348687.0,348997.0,349231.0,349315.0,349389.0,349430.0,349450.0,349454.0
20,31444.0,177331.0,269352.0,307352.0,325073.0,334864.0,340170.0,343208.0,344853.0,346333.0,347395.0,348010.0,348513.0,348916.0,349170.0,349326.0,349369.0,349410.0,349420.0
25,17716.0,148490.0,250762.0,296665.0,317960.0,330209.0,337956.0,341984.0,344080.0,345682.0,346962.0,347554.0,348135.0,348614.0,349070.0,349193.0,349317.0,349366.0,349394.0
30,9783.0,122788.0,234438.0,286311.0,311875.0,324770.0,333603.0,339443.0,342926.0,344989.0,346246.0,347343.0,347810.0,348285.0,348732.0,349007.0,349189.0,349331.0,349363.0


In [10]:
totalnum = weight["weight"].sum()
df2[["2","3","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20"]].div(totalnum)

,2,3,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Number of Restaurants,,,,,,,,,,,,,,,,,,,,
1,0.812068,0.938808,0.938808,0.973589,0.986500,0.992758,0.995839,0.998091,0.999047,0.999545,0.999771,0.999845,0.999914,0.999954,0.999974,0.999977,0.999989,0.999994,0.999994,1.000000
5,0.494818,0.822778,0.822778,0.918079,0.956162,0.974390,0.983893,0.989836,0.993670,0.996249,0.998097,0.998818,0.999442,0.999645,0.999794,0.999877,0.999920,0.999946,0.999969,0.999971
10,0.282180,0.715637,0.715637,0.870539,0.929193,0.958294,0.974739,0.983867,0.988966,0.993175,0.995794,0.997780,0.998916,0.999350,0.999657,0.999777,0.999863,0.999931,0.999951,0.999951
15,0.156167,0.607557,0.607557,0.822040,0.904608,0.944905,0.965479,0.978651,0.984983,0.989581,0.993407,0.995754,0.997754,0.998641,0.999310,0.999551,0.999762,0.999880,0.999937,0.999948
20,0.089976,0.507425,0.507425,0.770740,0.879475,0.930183,0.958200,0.973383,0.982076,0.986783,0.991018,0.994057,0.995817,0.997256,0.998409,0.999136,0.999582,0.999705,0.999823,0.999851
25,0.050694,0.424898,0.424898,0.717545,0.848895,0.909830,0.944880,0.967047,0.978573,0.984571,0.989155,0.992818,0.994512,0.996174,0.997545,0.998850,0.999202,0.999556,0.999697,0.999777
30,0.027994,0.351353,0.351353,0.670835,0.819267,0.892418,0.929316,0.954591,0.971302,0.981269,0.987172,0.990769,0.993908,0.995244,0.996603,0.997883,0.998669,0.999190,0.999597,0.999688


## Non-weighted version
Here we use another way to replicate the table.


In [11]:
result = pivot_cumsum.apply(pd.value_counts).fillna(0)
result["index"]=result.index
result

porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,index
0.0,6279.0,3203.0,1782.0,1052.0,612.0,403.0,241.0,145.0,85.0,52.0,33.0,22.0,12.0,9.0,8.0,4.0,2.0,2.0,0.0,0.0
1.0,1623.0,1125.0,683.0,517.0,365.0,223.0,156.0,90.0,61.0,30.0,18.0,10.0,8.0,3.0,4.0,4.0,1.0,1.0,3.0,1.0
2.0,994.0,763.0,548.0,329.0,267.0,195.0,133.0,80.0,54.0,33.0,10.0,7.0,5.0,4.0,1.0,2.0,1.0,0.0,0.0,2.0
3.0,782.0,570.0,479.0,354.0,244.0,195.0,164.0,132.0,101.0,73.0,53.0,24.0,18.0,10.0,7.0,5.0,3.0,3.0,2.0,3.0
4.0,607.0,441.0,325.0,223.0,180.0,134.0,103.0,83.0,44.0,37.0,41.0,30.0,16.0,12.0,9.0,7.0,6.0,1.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3529.0
3534.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3534.0
3542.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3542.0
3544.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3544.0


In [0]:
column1=result[result["index"]>=1].sum()
column5=result[result["index"]>=5].sum()
column10=result[result["index"]>=10].sum()
column15=result[result["index"]>=15].sum()
column20=result[result["index"]>=20].sum()
column25=result[result["index"]>=25].sum()
column30=result[result["index"]>=30].sum()
aggregate=pd.concat([column1, column5, column10, column15, column20, column25, column30], axis=1)
aggregate.columns = [1,5,10,15,20,25,30]
aggregate=aggregate.T

In [13]:
aggregate[drivingTimesinMin].div(totaldistinct)

porygon_dt,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,0.555406,0.773207,0.873823,0.925512,0.956666,0.971465,0.982936,0.989733,0.993981,0.996318,0.997663,0.998442,0.999150,0.999363,0.999434,0.999717,0.999858,0.999858,1.000000
5,0.271755,0.567939,0.729732,0.824754,0.881895,0.918573,0.943567,0.962473,0.975572,0.984069,0.989025,0.993415,0.995822,0.997309,0.997947,0.998442,0.999080,0.999504,0.999575
10,0.141542,0.450188,0.644410,0.760603,0.836225,0.887913,0.918997,0.941231,0.959569,0.972810,0.981944,0.989237,0.992990,0.995893,0.997097,0.997734,0.998725,0.999150,0.999150
15,0.074488,0.357714,0.575586,0.713588,0.800326,0.858953,0.900375,0.925299,0.945054,0.962402,0.973872,0.983573,0.988459,0.992990,0.995539,0.997026,0.998017,0.998867,0.999080
20,0.041209,0.284076,0.513064,0.667210,0.769312,0.836154,0.884727,0.914537,0.933796,0.953126,0.966013,0.976421,0.983431,0.989025,0.993132,0.995681,0.996743,0.997734,0.998230
25,0.022658,0.229979,0.459180,0.623522,0.728670,0.807973,0.865326,0.901650,0.925016,0.945125,0.959569,0.971182,0.979608,0.986193,0.991432,0.993557,0.995681,0.997168,0.997734
30,0.012108,0.183672,0.413510,0.583162,0.699214,0.778659,0.838774,0.883736,0.914112,0.937053,0.953055,0.966438,0.974793,0.982157,0.988459,0.991574,0.993981,0.996106,0.997239


## Kepler Visualisation


In [0]:
# %%capture help hide the installing process
%%capture   
!pip install keplergl

In [0]:
from keplergl import KeplerGl 

In [0]:
kepler = pivot.reset_index()
kepler = kepler.fillna(0)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0


In [0]:
col_list = list(kepler)
del col_list[0:3]
col_list

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [0]:
kepler["total_restaurant"]=kepler[col_list].sum(axis=1)
kepler

porygon_dt,geohash_center,geohash,weight,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,total_restaurant
0,POINT(121.343307495117 25.0440216064453),wsqq3pp,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,4.0,4.0,10.0,16.0,31.0,34.0,40.0,25.0,24.0,14.0,204.0
1,POINT(121.343307495117 25.0838470458984),wsqq9nr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,2.0,4.0,11.0,23.0,39.0,25.0,8.0,11.0,26.0,43.0,197.0
2,POINT(121.343307495117 25.0865936279297),wsqq9nz,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,5.0,15.0,25.0,35.0,23.0,7.0,12.0,26.0,154.0
3,POINT(121.344680786133 25.0399017333984),wsqq3q2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,7.0,14.0,6.0,31.0,42.0,39.0,43.0,5.0,7.0,196.0
4,POINT(121.344680786133 25.0412750244141),wsqq3q8,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,7.0,14.0,7.0,35.0,41.0,42.0,30.0,9.0,15.0,204.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14118,POINT(121.674270629883 25.0824737548828),wsqw8y0,6.0,0.0,0.0,2.0,0.0,2.0,0.0,6.0,17.0,3.0,18.0,18.0,16.0,25.0,8.0,12.0,4.0,7.0,3.0,7.0,148.0
14119,POINT(121.675643920898 25.0797271728516),wsqw8v9,10.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,6.0,14.0,9.0,18.0,17.0,16.0,24.0,11.0,7.0,6.0,7.0,2.0,141.0
14120,POINT(121.675643920898 25.0811004638672),wsqw8vc,16.0,0.0,0.0,1.0,0.0,2.0,1.0,0.0,9.0,15.0,8.0,20.0,15.0,15.0,24.0,11.0,8.0,3.0,8.0,3.0,143.0
14121,POINT(121.675643920898 25.0838470458984),wsqw8y3,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,11.0,9.0,13.0,19.0,12.0,23.0,19.0,9.0,10.0,5.0,5.0,3.0,142.0


In [0]:
!pip install pydeck

In [0]:
import pydeck

In [0]:
map_1 = KeplerGl()
#map_1.add_data(data=kepler, name='data_1')
!pip install ipython

import IPython

IPython.display.HTML(map_1)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


TypeError: ignored

In [0]:
map_1

KeplerGl()